<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Тестовое задание 3: Найти причину отсутствия заявок.

Исследуйте просмотры страниц сайта (см. файлы).
Предположите причину отсутствия заявок с сайта в период с 21 по 25 ноября.

Описание данных и проекта:
- Данные основаны на выгрузке из Яндекс Метрика Logs API.
- Цель "1234" - это цель "Заявка с сайта".
- Сайт застройщика продаёт квартиры в новостройке.
    - Страница "/mortgage" - это страница с описанием вариантов ипотеки.
    - Страница "/search" - это страница выбора квартир.
    - Страницы "/search?x=y" - это страница выбора квартир, когда пользователь выбрал какие-то параметры.
    - Страницы "/search/room-3k-XXX" - это страницы с подробным описанием квартир.
    - Страница "/other" - это какие-то другие страницы.

 <a id='plan'></a>

План исследования:
1. [Загрузить и подготовить данные](#load)
2. [Провести исследовательский анализ](#eda)
3. [Выдвинуть и проверить гипотезы](#hypotheses)
4. [Сделать выводы и дать рекомендации](#conclusion)

 <a id='load'></a>
# Загрузим и изучим данные

In [1]:
# импорт библиотек
import pandas as pd

In [2]:
# запись файловых путей
visits_path = '/Users/sergio/Downloads/'
hits_path = '/Users/sergio/Downloads/'

In [3]:
# чтение файлов
visits_data = pd.read_excel(visits_path + 'test__visits.xlsx')
hits_data = pd.read_excel(hits_path + 'test__hits.xlsx')


In [4]:
# таблица пользователей
visits_data.sample(5)

,ym:s:visitID,ym:s:goalsID
136390,6982174062365900800,[]
7605,6696415679393038336,[]
11051,6581442652040593408,[]
62568,6715793458114592768,[]
64725,6718655607826546688,[]


In [5]:
# сводная ифнормация таблицы пользователей
visits_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193016 entries, 0 to 193015
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ym:s:visitID  193016 non-null  int64 
 1   ym:s:goalsID  193016 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.9+ MB


In [6]:
# переименуем столбцы для удобства
visits_data.columns = ['visit_id', 'goal_id']

In [7]:
# таблица логов событий сайта
hits_data.head()

,ym:s:visitID,ym:pv:watchID,ym:pv:counterUserIDHash,ym:pv:date,ym:pv:dateTime,ym:pv:URL,ym:pv:referer
0,6560323411888570368,6560323843815112704,5962304341763423232,2024-11-04,2024-11-04 18:34:09,test.test/,test.test/
1,6560323411888570368,6560323844096655360,5962304341763423232,2024-11-04,2024-11-04 18:34:09,test.test/,test.test/
2,6560325594846330880,6560325594846330880,7277754749516948480,2024-11-04,2024-11-04 18:34:16,test.test/,NaN
3,6560325594846330880,6560326270037786624,7277754749516948480,2024-11-04,2024-11-04 18:34:17,test.test/,test.test/
4,6560325594846330880,6560326270366515200,7277754749516948480,2024-11-04,2024-11-04 18:34:17,test.test/,test.test/


In [8]:
# сводная ифнормация логов событий сайта
hits_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423317 entries, 0 to 423316
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ym:s:visitID             423317 non-null  int64         
 1   ym:pv:watchID            423317 non-null  int64         
 2   ym:pv:counterUserIDHash  423317 non-null  uint64        
 3   ym:pv:date               423317 non-null  datetime64[ns]
 4   ym:pv:dateTime           423317 non-null  datetime64[ns]
 5   ym:pv:URL                423317 non-null  object        
 6   ym:pv:referer            286290 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(2), uint64(1)
memory usage: 22.6+ MB


In [9]:
# переименуем столбцы для удобства
columns = ['visit_id', 'watch_id', 'user_hash_id', 'date', 'date_time', 'url', 'referer']
hits_data.columns=columns

[Вернуться к плану](#plan)

 <a id='eda'></a>
# Проведем исследовательский анализ

In [10]:
# определим целевые даты
start_date = '2024-11-21'
end_date = '2024-11-25'

# отфильтруем данные
hits_filtered = hits_data[(hits_data['date'] >= start_date) & (hits_data['date'] <= end_date)]
visits_filtered = visits_data[visits_data['visit_id'].isin(hits_filtered['visit_id'])]

# проверим сколько раз совершено целевое действие
goal_id = "1234"
goal_counts = visits_filtered['goal_id'].apply(lambda x: goal_id in x).sum()

goal_counts

0

По условию задачи целевое действие не было произведено, что мы только что подтвердили

In [11]:
# посмотрим сколько раз какие страныцы открывали
hits_filtered['url'].value_counts()

url
test.test/                         27423
test.test/search/                  18598
test.test/search/?x=y               5347
test.test/other                     2920
test.test/project/                  2487
test.test/search/?x=y&rooms=3к      1730
test.test/search/?x=y&rooms=2к      1679
test.test/search/room-2kom-XXX/     1268
test.test/search/room-3kom-XXX/     1221
test.test/mortgage/                  517
test.test/contacts/                  130
Name: count, dtype: int64

[Вернуться к плану](#plan)

 <a id='hypotheses'></a>
# Выдвинем и проверим гипотезы

**Первая гипотеза: пользователи не оставляли заявки, потому что не доходили до страницы с контактами.**

130 посещений за 5 дней, при условии, что на другие страницы заходили. 

Проверим сколько заходили на страницы, которые потенциально могли привести к заявке: просмотр квартиры и условий ипотеки 

Посетителей страниц **/mortgage и /search/room...** будем считать уникальных, осальные страницы нам не так интересны, потому что они не дают точного понимания, что пользователь ознакомился с информацией о квартире или ипотеке. 

In [12]:
# фильтруем визиты по искомым признакам
mortgage_visits = hits_filtered[hits_filtered['url'].str.contains('/mortgage')]

room_visits = hits_filtered[hits_filtered['url'].str.contains('/search/room')]


In [13]:
# считаем все заходы на страницу контактов
contacts_visits = hits_filtered[hits_filtered['url'].str.contains('/contacts')]

# считаем все заходы на сайт
total_visits = hits_filtered[~hits_filtered['url'].str.contains('/contacts')]

# конверсия в переход на страницу контактов из всех заходов на сайт
round(len(contacts_visits) / len(total_visits) * 100, 2)

0.21

In [14]:
# считаем длину заходов на страницу с информацией об ипотеке
unique_mortgage_visits = len(mortgage_visits['visit_id'].unique())

# считаем длину заходов на страницу с подробной информацией о квартирах
unique_room_visits = len(room_visits['visit_id'].unique())

# конверсия в переход на страницу контактов из всех заинтересованных пользователей
round((unique_mortgage_visits + unique_room_visits) / len(contacts_visits), 2)

10.22

Конверсия со всех заходов очень маленькая - всего 0.21%, однако конверсия потенициально заинтересованных пользователей 10%. Возможно, люди приходили и не могли пройти на этап дальше по техническим проблемам. 

**Рекомендация обратить внимание на тех. обслуживание сайта и поддержку работоспособности, а также высоконагруженности**

Еще одна гипотеза - проверить с каких страниц пользователи приходили на страницу с контактами.

In [15]:
contacts_visits['referer'].value_counts()

referer
test.test/               34
test.test/contacts/      31
yyy.xx                   30
test.test/search/?x=y     9
test.test/project/        7
test.test/search/         5
test.test/other           3
Name: count, dtype: int64

Всего 9 человек пришло со страницы фильтра квартир и 31 пользователь пришел с этой же страницы с контактами. 

Возможно, на страницах с информацией об ипотеке и на страницах с подробной информацией о квартирах просто не было кнопки-перехода на страницу контактов, чтобы пользователь смог подать заявку. Возможно, по технической причине. После обновления сайта или выкатывания нового куска кода. 

**Рекомендация: проверять что все кнопки для целевых дейсвтий всегда на сайта и работают**

[Вернуться к плану](#plan)

 <a id='conclusion'></a>
# Выводы и рекомендации

Конверсия со всех заходов остаётся крайне низкой, однако среди потенциально заинтересованных пользователей она достигает 10%. Это может свидетельствовать о технических проблемах, мешающих пользователям пройти дальше по воронке.

**Рекомендация: уделить внимание техническому обслуживанию сайта, включая мониторинг его работоспособности и адаптацию к высоким нагрузкам.**

Анализ источников трафика показывает, что всего 9 человек перешли со страницы фильтра квартир, тогда как 31 пользователь попал на страницу с контактами с того же источника.

Это может указывать на отсутствие кнопки или другого элемента для перехода на страницу контактов на страницах с информацией об ипотеке или подробностями о квартирах. Возможной причиной могло быть техническое упущение — например, ошибки при выкладке обновлений сайта или нового функционала.

**Рекомендация: регулярно проверять наличие и корректную работу всех кнопок и элементов, необходимых для совершения целевых действий на сайте.**

[Вернуться к плану](#plan)